In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
from time import time
from scipy.special import erf
from LumFunc_miniJPAS import LumFunc_hist

In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_30deg.npy')
pm_data_noerr = np.load('npy/pm_LAE_30deg.npy')
f_name = 'Source_cats/Source_cat_LAE_30deg.npy'
mock = np.load(f_name, allow_pickle=True).item()

errors = np.load('npy/median_errors.npy')

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
nb_c = 13
N_nb = 6

pm_err = (np.ones(pm_data.shape).T * errors).T
cont_est, cont_err = stack_estimation(
    pm_data, pm_err, nb_c, N_nb, w_central
    )
real_cont_value, _ = synthetic_BB_estimation(
    pm_data_noerr, pm_err, nb_c, 1, tcurves['t'], tcurves['w'], w_central, False
    )

In [ ]:
## Plots
np.random.seed(22)
for mock_n in np.random.randint(0, pm_data.shape[1], 20):
    pm = pm_data[:,mock_n]
    
    fig, ax = plt.subplots(figsize = (12, 8))
    ax.plot(mock['w_Arr'], mock['SEDs_no_line'][mock_n], c='orange', lw=2, zorder=-1)
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], c='gray', fmt='.')
    ax.scatter(w_central[nb_c], real_cont_value[mock_n], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4],
                xerr=bb_fwhm[-4]/2, yerr=errors[-4],
                fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], yerr=errors[-3],
                xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], yerr=errors[-2],
                xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], yerr=errors[-1],
                xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    # Fit line
    ax.errorbar(w_central[nb_c], cont_est[mock_n], yerr=cont_err[mock_n],
                c='violet', marker='^', markersize=9,
                capsize=4, label='Stack NBs', elinewidth=2, capthick=2)
    
    plt.legend()

    plt.show()
    print(mock['EW_Arr'][mock_n])

In [ ]:
fig, ax = plt.subplots(2, figsize=(13, 10), sharex=True)
for N_nb in np.arange(1, 21):
    pm_err = (np.ones(pm_data.shape).T * errors).T

    cont_est_stk, cont_err_stk = stack_estimation(
        pm_data, pm_err, nb_c, N_nb, w_central
    )

    real_cont_value, _ = synthetic_BB_estimation(
        pm_data_noerr, pm_err, nb_c, 2, tcurves['t'], tcurves['w'], w_central, False
        )
    
    mean_stk = np.average(((cont_est_stk - real_cont_value) / real_cont_value))
    abs_mean_stk = np.average(np.abs((cont_est_stk - real_cont_value) / real_cont_value))

    ax[0].bar(N_nb, mean_stk, color='C1')
    ax[1].bar(N_nb, abs_mean_stk, color='C1')
plt.subplots_adjust(hspace=.0)
ax[0].set_xticks(np.arange(1, 21))
plt.plot()

In [ ]:
fig, ax =  plt.subplots(figsize = (13, 10))

N_points = 50
sigma_min = 0.01
sigma_max = 7

sigma_Arr = np.linspace(sigma_min, sigma_max, N_points)
sigma_hist_stk = np.zeros(N_points)

ax.plot(erf(sigma_Arr / 2**0.5), erf(sigma_Arr / 2**0.5), linestyle='dashed', c='C1')

N_sources = pm_data.shape[1]

for nb_c in [7, 8, 9, 10, 11, 12, 13, 14, 30]:
    N_nb_stack = 6
    err = (np.ones(pm_data.shape).T * errors).T
    cont_stk, cont_err_stk = stack_estimation(
        pm_data, err, nb_c, N_nb_stack, w_central
        )
    real_cont_value, _ = synthetic_BB_estimation(
        pm_data_noerr, pm_err, nb_c, 1, tcurves['t'], tcurves['w'], w_central, False
    )
    ## Synthetic photometry of the continuum using the central NB
    w_t = tcurves['w'][nb_c]
    t = tcurves['t'][nb_c]
    w_Arr = mock['w_Arr']

    #### ERROR TESTING ####

    sigma_hist = np.zeros(N_points)

    for i in range(N_points):
        sigma_hist_stk[i] = len(
            np.where(
                np.abs(cont_stk - real_cont_value) < sigma_Arr[i] * cont_err_stk
            )[0]
        )

    # Plot the error test
    ax.set_ylabel('N / N_sources', fontsize=15)
    ax.set_xlabel('$\sigma$', fontsize=15)

    ax.plot(
        erf(sigma_Arr / 2**0.5), sigma_hist_stk/N_sources, marker='.',
        linestyle='none', label=filters_tags[nb_c]
    )    

ax.legend(fontsize=15)
ax.set_xticks(np.linspace(0, 1, 11))
ax.set_yticks(np.linspace(0, 1, 11))
ax.grid()
plt.show()

In [ ]:
## For a NB make the error test but picking sources according to their distance
## to the real redshift
fig, ax =  plt.subplots(figsize = (13, 10))

N_points = 50
sigma_min = 0.01
sigma_max = 7

nb_c = 10

sigma_Arr = np.linspace(sigma_min, sigma_max, N_points)
sigma_hist_stk = np.zeros(N_points)

ax.plot(erf(sigma_Arr / 2**0.5), erf(sigma_Arr / 2**0.5), linestyle='dashed', c='C1')

N_sources = pm_data.shape[1]

z_nb_Arr = np.array(w_central) / 1215.67 - 1
line_pos_Arr = np.zeros(N_sources)
for i in range(N_sources):
    line_pos_Arr[i] = np.argmin(np.abs(mock['redshift_Lya_Arr'][i] - z_nb_Arr[:-4]))

print(line_pos_Arr)

for dist in [(0,0), (1,1), (2,2), (5,99)]:
    N_nb_stack = 9

    mask = (
        (np.abs(line_pos_Arr - nb_c) >= dist[0])
        & (np.abs(line_pos_Arr - nb_c) <= dist[1])
    )

    err = (np.ones(pm_data.shape).T * errors).T
    cont_stk, cont_err_stk = stack_estimation(
        pm_data[:, mask], err[:, mask], nb_c, N_nb_stack, w_central
        )
    real_cont_value, _ = synthetic_BB_estimation(
        pm_data_noerr[:, mask], pm_err[:, mask], nb_c, 1, tcurves['t'],
        tcurves['w'], w_central, False
    )
    ## Synthetic photometry of the continuum using the central NB
    w_t = tcurves['w'][nb_c]
    t = tcurves['t'][nb_c]
    w_Arr = mock['w_Arr']

    #### ERROR TESTING ####

    sigma_hist = np.zeros(N_points)

    for i in range(N_points):
        sigma_hist_stk[i] = len(
            np.where(
                np.abs(cont_stk - real_cont_value) < sigma_Arr[i] * cont_err_stk
            )[0]
        )

    # Plot the error test
    ax.set_ylabel('N / N_sources', fontsize=15)
    ax.set_xlabel('$\sigma$', fontsize=15)

    N_sources_masked = len(np.where(mask)[0])

    ax.plot(
        erf(sigma_Arr / 2**0.5), sigma_hist_stk/N_sources_masked, marker='.',
        linestyle='none', label=dist
    )    

ax.legend(fontsize=15)
ax.set_xticks(np.linspace(0, 1, 11))
ax.set_yticks(np.linspace(0, 1, 11))
ax.grid()
plt.show()

In [ ]:
## Load HETDEX LumFunc data (Zhang 2021)
filepath = 'csv/HETDEX_LumFunc.csv'
HETDEX_LF = []
with open(filepath, mode='r') as csvfile:
    rdlns = csv.reader(csvfile, delimiter=',')
    for line in rdlns:
        HETDEX_LF.append(line)
HETDEX_LF = np.array(HETDEX_LF).astype(float)

In [ ]:
## Retrieved LF
fig, ax = plt.subplots(figsize=(12, 9))
for nb_c in np.arange(9, 14):
    N_nb_stack = 7
    cont_stk, cont_err_stk = stack_estimation(
            pm_data, err, nb_c, N_nb_stack, w_central
            )
    
    ew0min = 30
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(tcurves, nb_c, True)
    diff_err = (errors[nb_c]**2 + cont_err_stk**2)**0.5
    f_line = pm_data[nb_c] - cont_stk
    line = (
        (pm_data[nb_c] - cont_stk > 3*diff_err)
        & (pm_data[nb_c] - cont_stk > ew0min * (1 + z_nb) * cont_stk / fwhm)
    )
    f_line = f_line[line]
    x, lf_hist, vol, binw = LumFunc_hist(f_line, w_central[nb_c], fwhm, 10, obs_area=30,
        L_min=43.5, L_max=45
    )
    y = lf_hist / vol / binw

    ax.errorbar(x, y, marker='.', markersize=12, linestyle='none',
        yerr=np.array(lf_hist)**0.5/vol/binw, label=filters_tags[nb_c]
    )

    #ax.set_xlim((43.8, 45.25))
    # ax.set_ylim((0, 0.6e-5))
    ax.set_yscale('log')
ax.plot(HETDEX_LF[:,0], HETDEX_LF[:,1], 's', markersize = 10, label = 'Mock input')

out_LF_hist, out_LF_bins = np.histogram(mock['L_line'])
out_vol = z_volume(2.25, 3.2, 30)
x_out = [(out_LF_bins[i+1] + out_LF_bins[i]) / 2 for i in range(len(out_LF_bins) - 1)]
y_out = out_LF_hist / out_vol / (out_LF_bins[1] - out_LF_bins[0])
ax.plot(x_out, y_out, marker='x', c='black', linestyle='none', label='Mock output')

ax.legend(fontsize=15)
ax.set_title('Retrieved LF', fontsize=20)
plt.show()

In [ ]:
## EW distribution in and out
fig, ax = plt.subplots(figsize=(12,9))

ax.hist(mock['EW_Arr'], bins=np.linspace(0, 300, 25), density=True, label='Output EW$_0$')
ew_x = np.linspace(0, 300, 1000)
w0 = 75
ew_norm = 1. / (-w0 * (np.exp(-500/w0) - np.exp(-10/w0)))
ax.plot(ew_x, np.exp(-ew_x/w0) * ew_norm, label='Input EW$_0$ distribution')

ew_ret_hist = np.zeros(14)

for nb_c in range(9, 14):
    N_nb_stack = 7
    cont_stk, cont_err_stk = stack_estimation(
            pm_data, err, nb_c, N_nb_stack, w_central
            )
    
    ew0min = 30
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(tcurves, nb_c, True)
    diff_err = (errors[nb_c]**2 + cont_err_stk**2)**0.5
    f_line = pm_data[nb_c] - cont_stk
    line = (
        (pm_data[nb_c] - cont_stk > 3*diff_err)
        & (pm_data[nb_c] - cont_stk > ew0min * (1 + z_nb) * cont_stk / fwhm)
    )
    ew_retrieved = pm_data[nb_c, line] * fwhm / cont_stk[line] / (1 + z_nb)
    aux1, ew_ret_bins = np.histogram(ew_retrieved, bins=np.linspace(0, 300, 15))
    ew_ret_hist += aux1

ax.plot([(ew_ret_bins[i] + ew_ret_bins[i+1]) / 2 for i in range(len(ew_ret_hist))],
    ew_ret_hist / (np.sum(ew_ret_hist) * np.diff(ew_ret_bins)),
    marker='s', markersize=10, linestyle='--', label='Retrieved EW$_0$'
    )

ax.set_xlabel('EW$_0$ ($\AA$)', fontsize=15)
ax.set_ylabel('N / dEW$_0$', fontsize=15)
# ax.set_xlim((0,300))
ax.legend(fontsize=15)
plt.show

In [ ]:
print((w_central[13] - nb_fwhm(tcurves, 13, True)/2) / 1215.67 - 1,
    (w_central[13] + nb_fwhm(tcurves, 13, True)/2) / 1215.67 - 1
)